In [1]:
import struct

In [2]:
_format1 = ">4si"
_format2 = ">iqqiiiBiqi18s"

In [3]:
name = input("Enter name of the ROOT file - ")
if name.endswith(".root") is False:
    name = name + ".root"
file = open(name, "w+b")

In [4]:
fVersion = 61400
begin = struct.pack(_format1, b"root", fVersion)
file.write(begin)

8

In [5]:
#constants for file sample-6.14.00-lz4.root
fBegin = 100
fEnd = 50903
fSeekFree = 50836
fNbytesFree = 67
nfree = 1
fNbytesName = 82
fUnits = 4
fCompress = 404
fSeekInfo = 45408
fNbytesInfo = 5327
fUUID = b'\x00\x01<9\x994t^\x11\xe8\x80\x11\x01\x00\x00\x7f\xbe\xef'
start = struct.pack(_format2, fBegin, fEnd, fSeekFree, fNbytesFree, nfree, fNbytesName, fUnits, fCompress, fSeekInfo, fNbytesInfo, fUUID)
file.write(start)

67

In [6]:
#Test
import os
statinfo = os.stat('rootpy.root')
statinfo.st_size

0

In [ ]:
#Test
import uproot
uproot.open('~/Downloads/uproot/rootpy.root')